In [1]:
from modules.resnet import ResNet
import numpy as np
import tensorflow as tf
from modules import feature_extractor as fe
from modules import modelCreator
from modules.dataset_selecter import datasetSelector
from sklearn.model_selection import StratifiedKFold

import os
import random
from tensorflow.python.eager import context
from tensorflow.python.framework import ops

seed_value = 56

os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)
np.random.RandomState(seed_value)
np.random.seed(seed_value)
context.set_global_seed(seed_value)
ops.get_default_graph().seed = seed_value

os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
np.random.seed(seed_value)


def run_experiments(seed_value=56, 
                    takeName=False, use_cache=True, saveMethod='pickle',
                    skipDebugSaves=True, saveWeights=False,
                    nrFolds=5, batchSize=100, numEpochs=500, patience=70,
                    numbers=[1], 
                    method='TSFRESH_SEL_SYM', symbolCount=0, strategy=None, ncoefI=None, coef_div=None, 
                    architecture='transformer', numOfAttentionLayers=None, header=None, dmodel=None, dff=None, dropOutRate=None, lr=None, warmup=None, 
                    num_resblocks=None, num_channels=None, use_1x1conv=None):
    
    for number in numbers:

        kf = StratifiedKFold(nrFolds, shuffle=True, random_state=seed_value)

        X_train, X_test, y_train, y_test, y_trainy, y_testy, seqSize, dataName, num_of_classes, number = datasetSelector('ucr', seed_value, number, takeName, use_cache)


        saved = fe.save_foldwise_features(number, dataName, method, X_train, X_test, y_train, y_test, y_trainy, y_testy, kf)
        fold = 0

        abstractionString = method
        fullResults = dict()

        accuracies = []

        for train, test in kf.split(X_train, y_trainy):
            fold += 1
            print("Starting fold: ", fold)

            x_train1, x_val, x_test, y_train1, y_val, y_test, y_trainy1, y_valy, y_testy = fe.load_foldwise_features(number, dataName, method, fold, saveMethod)
                
            if np.any(np.isnan(x_train1.flatten())) or np.any(np.isnan(x_val.flatten())) or np.any(np.isnan(x_test.flatten())) or np.any(np.isnan(y_train1.flatten())) or np.any(np.isnan(y_val.flatten())) or np.any(np.isnan(y_test.flatten())):
                continue

            if x_train1 is None:
                print("No relevant features extracted, skipping..")
                continue 
            
            x_train1, x_val, x_test, y_train1, y_val, y_test, X_train_ori, X_val_ori, X_test_ori, y_trainy1, y_testy = modelCreator.preprocessData(x_train1, x_val, x_test, y_train1, y_val, y_test, 
                                                                                                                                                y_trainy1, y_testy, fold, symbolsCount=symbolCount, dataName=dataName, 
                                                                                                                                                seqSize=seqSize,
                                                                                                                                                strategy=strategy,
                                                                                                                                                method=method,
                                                                                                                                                ncoef=ncoefI, coef_div=coef_div)

            out = modelCreator.doAbstractedTraining(x_train1, x_val, x_test, y_train1, y_val, y_testy, 
                                                    BATCH=batchSize, seed_value=seed_value, num_epochs=numEpochs, patience = patience, 
                                                    num_of_classes=num_of_classes, number=number, dataName=dataName, fold=fold,
                                                    architecture=architecture, symbolCount=symbolCount, 
                                                    numOfAttentionLayers=numOfAttentionLayers,header=header, dmodel=dmodel, dff=dff, 
                                                    num_resblocks=num_resblocks, num_channels=num_channels, use_1x1conv=use_1x1conv,
                                                    abstractionType=abstractionString, rate=dropOutRate, lr=lr, warmup=warmup, 
                                                    skipDebugSaves=skipDebugSaves, saveWeights=saveWeights)
            
            saveName = modelCreator.getWeightName(number, dataName, 0,
                                                    learning = False, results = True,
                                                    architecture=architecture, abstractionType=method, symbols=symbolCount,
                                                    layers=numOfAttentionLayers, header=header, dmodel=dmodel, dff=dff,
                                                    num_resblocks=num_resblocks, num_channels=num_channels, use_1x1conv=use_1x1conv,
                                                    num_coef=ncoefI, dropout=dropOutRate, strategy=strategy)

            
            print("finished fold: " + str(fold))
            print("architecture")
            print("Method: ", method)
            print("Dataset: ", number)
            print("Fold: ", fold)
            print("Accuracy: ", out[1][0])
            accuracies.append(out[1][0])
            print(saveName)

        print("Final results")
        print("Method: ", method)
        print("Dataset: ", number)
        print("Average accuracy: ", np.mean(accuracies))

# dataset parameters
takeName = False
use_cache = True
saveMethod = 'pickle'

# additional saves
skipDebugSaves = True
saveWeights = False

# training parameters
nrFolds = 5
batchSize = 100
numEpochs = 500
patience = 70

# dataset numbers
numbers = [1]

# method
method = 'TSFRESH_SEL_SYM'
symbolCount = 5
strategy = 'uniform'
ncoefI = 0 # needed for SFA
coef_div = 0 # needed for SFA

# architecture 'transformer' or 'resnet'

### transformer model parameters
architecture = 'transformer'
numOfAttentionLayers = 3
header = 4
dmodel = 32
dff = 16
dropOutRate = 0.0
lr = 'custom'
warmup = 10000

## resnet model parameters
# architecture = 'resnet'
# num_resblocks = 3
# num_channels = 16
# use_1x1conv = True
# lr = 0.0001

run_experiments(seed_value=seed_value, 
                takeName=takeName, use_cache=use_cache, saveMethod=saveMethod,
                skipDebugSaves=skipDebugSaves, saveWeights=saveWeights,
                nrFolds=nrFolds, batchSize=batchSize, numEpochs=numEpochs, patience=patience,
                numbers=numbers, 
                method=method, symbolCount=symbolCount, strategy=strategy, ncoefI=ncoefI, coef_div=coef_div, 
                architecture=architecture, 
                numOfAttentionLayers=numOfAttentionLayers, header=header, dmodel=dmodel, dff=dff, dropOutRate=dropOutRate, lr=lr, warmup=warmup)
                # num_resblocks=num_resblocks, num_channels=num_channels, use_1x1conv=use_1x1conv, lr=lr)

2025-02-28 15:05:29.358127: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-28 15:05:29.489199: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740751529.536433 1152178 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740751529.549609 1152178 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-28 15:05:29.663895: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Num GPUs Available:  1
---------------------------------- SAVING FEATURES ----------------------------------
---------------------------------- FOLD 1 ----------------------------------
File already exists
---------------------------------- FOLD 2 ----------------------------------
File already exists
---------------------------------- FOLD 3 ----------------------------------
File already exists
---------------------------------- FOLD 4 ----------------------------------
File already exists
---------------------------------- FOLD 5 ----------------------------------
File already exists
Starting fold:  1


I0000 00:00:1740751536.810185 1152178 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21458 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9
2025-02-28 15:05:38.107699: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_15}}


1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step

2025-02-28 15:06:59.108310: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


...

1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 180ms/step

2025-02-28 15:10:36.993123: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
validation fold score with input TSFRESH_SEL_SYM(accuracy): 0.7692307692307693
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
test fold score with input TSFRESH_SEL_SYM-(accuracy): 0.7493606138107417
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
finished fold: 4
architecture
Method:  TSFRESH_SEL_SYM
Dataset:  1
Fold:  4
Accuracy:  0.7493606138107417
./results/results-1_Adiac -a transformer -m TSFRESH_SEL_SYM -s 5 -l 3 -h 4 -dm 32 -df 16 -dr 0.0 -st u -cf 0.h5
Starting fold:  5
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step

2025-02-28 15:11:39.260170: E tensorflow/core/framework/node_def_util.cc:676] NodeDef mentions attribute use_unbounded_threadpool which is not in the op definition: Op<name=MapDataset; signature=input_dataset:variant, other_arguments: -> handle:variant; attr=f:func; attr=Targuments:list(type),min=0; attr=output_types:list(type),min=1; attr=output_shapes:list(shape),min=1; attr=use_inter_op_parallelism:bool,default=true; attr=preserve_cardinality:bool,default=false; attr=force_synchronous:bool,default=false; attr=metadata:string,default=""> This may be expected if your graph generating binary is newer  than this binary. Unknown attributes will be ignored. NodeDef: {{node ParallelMapDatasetV2/_14}}


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 257ms/step
validation fold score with input TSFRESH_SEL_SYM(accuracy): 0.6666666666666666
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
test fold score with input TSFRESH_SEL_SYM-(accuracy): 0.7391304347826086
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
finished fold: 5
architecture
Method:  TSFRESH_SEL_SYM
Dataset:  1
Fold:  5
Accuracy:  0.7391304347826086
./results/results-1_Adiac -a transformer -m TSFRESH_SEL_SYM -s 5 -l 3 -h 4 -dm 32 -df 16 -dr 0.0 -st u -cf 0.h5
Final results
Method:  TSFRESH_SEL_SYM
Dataset:  1
Average accuracy:  0.7268542199488491
